In [1]:
from rdkit import Chem
from rdkit.Chem import QED
import pandas as pd
from tqdm import tqdm
import time
from rdkit.Chem import RDConfig
import os
import sys
import math
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
# now you can import sascore!
import sascorer

In [2]:
def matrix(fname, prop_name):
    f =  open(fname,'r')
    lines = f.readlines()
    # print(len(lines))

    iupac_list = []
    fg_list = []
    for line in lines:
        if 'startofiupac' in line:
            # tmp = line.split('>')[1].strip()
            tmp = line.split('<|startofiupac|>')
            iupac_list.append(tmp[-1].strip())

            # break
        elif 'Reference' in line:
            tmp = line.split('fgs: ')[1].strip()
            # tmp = line.split('fgs: ')[1].strip().split('.<')[0].strip()
            fg_list.append(tmp)
        else:
            pass

    nums_iupac = len(iupac_list)
    nums_fg = len(fg_list)

    assert nums_iupac == nums_fg
    # print(nums)

    hits = 0
    hits_list = []
    unhits = 0
    success_list = []
    fail_list = []
    for i in range(nums_iupac):
        # if i % 5 != 0:
        #     continue
        true_idx = math.floor(i/25)

        fgs = fg_list[i].replace(' ','').split(',')
        iupac = iupac_list[i]

        for fg in fgs:
            if fg in iupac:
                hits = hits + 1
                hits_list.append(fg)
                success_list.append(true_idx)
            else:
                unhits = unhits + 1
                fail_list.append(true_idx)

        # for fg in fgs:
        #     if fg not in iupac:
        #         unhits = unhits + 1
        #         fail_list.append(true_idx)
        #         break
        # else:
        #     hits = hits + 1
        #     success_list.append(true_idx)

    hits_list = list(set(hits_list))
    # print(hits_list)
    total = hits + unhits

    sr = hits/total
    sr = round(sr, 4)

    success_list = list(set(success_list))
    fail_list = list(set(fail_list))

    true_success = [i for i in success_list if i not in fail_list]
    half_success = [i for i in success_list if i in fail_list]
    true_fail = [i for i in fail_list if i not in success_list]
    
    print('Num of iupac is: ', nums_iupac)
    print('Total number is: ', total)
    print(f'File is {fname}, SR is: {sr}')
    print('Len of success: ', len(success_list))
    print('Len of failed: ', len(fail_list))
    print('Len of true success: ', len(true_success))
    print('Len of half success: ', len(half_success))
    print('Len of true failed: ', len(true_fail))
    print('Len of hits list: ', len(hits_list))

    true_success.extend(half_success)
    outputs = pd.DataFrame(data=true_success, columns=['success'])
    outputs.to_csv(f'./data/success_{prop_name}.csv', index=False)

In [3]:
def matrix2(fname, prop_name, step=5, top=5):
    f =  open(fname,'r')
    lines = f.readlines()
    # print(len(lines))

    iupac_list = []
    fg_list = []
    for line in lines:
        if 'startofiupac' in line:
            # tmp = line.split('>')[1].strip()
            tmp = line.split('<|startofiupac|>')
            iupac_list.append(tmp[-1].strip())

            # break
        elif 'Reference' in line:
            tmp = line.split('fgs: ')[1].strip()
            fg_list.append(tmp)
        else:
            pass

    nums_iupac = len(iupac_list)
    nums_fg = len(fg_list)

    print('Num of iupac: ', nums_iupac)

    assert nums_iupac == nums_fg
    # print(nums)

    iupac_list_merge = []
    fg_list_merge = []
    for i in range(0, nums_iupac, step):
        iupac_merge = ''
        # iupac_merge = iupac_list[i]

        # for j in range(i, i+top):
        for j in range(i, i+top, 5):
            if j != i+top-1:
                assert fg_list[j] == fg_list[j+1]
            iupac_merge = iupac_merge + iupac_list[j]

        fg_list_merge.append(fg_list[i])
        iupac_list_merge.append(iupac_merge)

    nums_iupac_merge = len(iupac_list_merge)
    nums_fg_merge = len(fg_list_merge)

    assert nums_iupac_merge == nums_fg_merge

    print('Num of merged iupac: ', nums_iupac_merge)

    hits = 0
    unhits = 0
    success_list = []
    fail_list = []
    for i in range(nums_iupac_merge):
        # if i % 5 != 0:
        #     continue
        true_idx = math.floor(i/5)

        fgs = fg_list_merge[i].replace(' ','').split(',')
        iupac = iupac_list_merge[i]

        # print(fgs)

        for fg in fgs:
            if fg not in iupac:
                unhits = unhits + 1
                fail_list.append(true_idx)
                break
        else:
            success_list.append(true_idx)
            hits = hits + 1

    total = hits + unhits

    sr = hits/total
    sr = round(sr, 4)

    success_list = list(set(success_list))
    fail_list = list(set(fail_list))

    true_success = [i for i in success_list if i not in fail_list]
    half_success = [i for i in success_list if i in fail_list]
    true_fail = [i for i in fail_list if i not in success_list]
    
    print('Num of iupac is: ', nums_iupac)
    print('Total number is: ', total)
    print(f'File is {fname}, SR is: {sr}')
    print('Len of success: ', len(success_list))
    print('Len of failed: ', len(fail_list))
    print('Len of true success: ', len(true_success))
    print('Len of half success: ', len(half_success))
    print('Len of true failed: ', len(true_fail))

    true_success.extend(half_success)
    outputs = pd.DataFrame(data=true_success, columns=['success'])
    outputs.to_csv(f'./data/success_{prop_name}.csv', index=False)

In [26]:
fname = './log/eval_fgs_5000000_1_fg_5.log'
matrix2(fname, '1_fg')

fname = './log/eval_fgs_5000000_2_fg_5.log'
matrix2(fname, '2_fg')

fname = './log/eval_fgs_5000000_3_fg_5.log'
matrix2(fname, '3_fg')

Num of iupac:  250000
Num of merged iupac:  50000
Num of iupac is:  250000
Total number is:  50000
File is ./log/eval_fgs_5000000_1_fg_5.log, SR is: 0.6201
Len of success:  8890
Len of failed:  7473
Len of true success:  2527
Len of half success:  6363
Len of true failed:  1110
Num of iupac:  250000
Num of merged iupac:  50000
Num of iupac is:  250000
Total number is:  50000
File is ./log/eval_fgs_5000000_2_fg_5.log, SR is: 0.7223
Len of success:  9541
Len of failed:  6676
Len of true success:  3324
Len of half success:  6217
Len of true failed:  459
Num of iupac:  250000
Num of merged iupac:  50000
Num of iupac is:  250000
Total number is:  50000
File is ./log/eval_fgs_5000000_3_fg_5.log, SR is: 0.6357
Len of success:  9184
Len of failed:  7593
Len of true success:  2407
Len of half success:  6777
Len of true failed:  816


In [ ]:
fname = './log/eval_fgs_31000000_1_fg_4.log'
matrix(fname, '1_fg')

fname = './log/eval_fgs_31000000_2_fg_4.log'
matrix(fname, '2_fg')

fname = './log/eval_fgs_31000000_3_fg_4.log'
matrix(fname, '3_fg')

In [9]:
def filter(fname, fname2, fg_num):
    df = pd.read_csv(fname)
    data_list = df.values

    idx_df = pd.read_csv(fname2)
    idx_list = idx_df.values

    outputs = []
    for idx in idx_list:
        data = data_list[idx]
        outputs.append(data[0])

        if len(outputs) >= 1000:
            break
    
    outputs = pd.DataFrame(data=outputs, columns=['cid', 'iupac', 'desc', 'smiles'])
    outputs.to_csv(f'./data/eval/eval_f{fg_num}_fg.csv', index=False)
    print('Saved.')


In [27]:
# filter('./data/eval/eval_1_fg.csv', './data/success_1_fg.csv', 1)
filter('./data/eval/eval_2_fg.csv', './data/success_2_fg.csv', 2)
filter('./data/eval/eval_3_fg.csv', './data/success_3_fg.csv', 3)

Saved.
Saved.


In [67]:
def matrix_top(fname, step=25, top=25):
    f1 = './data/n1.csv'
    df1 = pd.read_csv(f1)

    f2 = './data/n1.csv'
    df2 = pd.read_csv(f2)

    df = pd.concat([df1,df2])

    train_smiles = df['smiles'].values


    f =  open(fname,'r')
    lines = f.readlines()
    # print(len(lines))

    iupac_list = []
    fg_list = []
    smiles_list = []
    for line in lines:
        if 'startofsmiles' in line:
            # tmp = line.split('>')[1].strip()
            tmp = line.split('<|startofiupac|>')
            iupac_list.append(tmp[-1].strip())

            smiles_list.append(tmp[0].strip().split('<|startofsmiles|>')[-1].strip())

            # break
        elif 'Reference' in line:
            tmp = line.split('fgs: ')[1].strip()
            fg_list.append(tmp)
        else:
            pass

    nums_iupac = len(iupac_list)
    nums_fg = len(fg_list)

    smiles_list = list(set(smiles_list))
    nums_smiles = len(smiles_list)

    print('Num of iupac: ', nums_iupac)


    assert nums_iupac == nums_fg
    # print(nums)

    iupac_list_merge = []
    fg_list_merge = []
    for i in range(0, nums_iupac, step):
        iupac_merge = ''
        # iupac_merge = iupac_list[i]

        # for j in range(i, i+top):
        for j in range(i, i+top, 5):
            if j != i+top-1:
                assert fg_list[j] == fg_list[j+1]
            iupac_merge = iupac_merge + iupac_list[j]

        fg_list_merge.append(fg_list[i])
        iupac_list_merge.append(iupac_merge)


    iupac_list_merge = iupac_list_merge[:2000]
    fg_list_merge = fg_list_merge[:2000]
    smiles_list = smiles_list[:2000]
    nums_smiles = len(smiles_list)

    # iupac_list_merge = iupac_list_merge
    # fg_list_merge = fg_list_merge

    nums_iupac_merge = len(iupac_list_merge)
    nums_fg_merge = len(fg_list_merge)

    assert nums_iupac_merge == nums_fg_merge

    print('Num of merged iupac: ', nums_iupac_merge)
    print('Num of smiles: ', nums_smiles)

    hits = 0
    unhits = 0
    for i in range(nums_iupac_merge):
        # if i % 5 != 0:
        #     continue

        fgs = fg_list_merge[i].replace(' ','').split(',')
        iupac = iupac_list_merge[i]

        # print(fgs)
        # for fg in fgs:
        #     if fg not in iupac:
        #         unhits = unhits + 1
        #         break
        # else:
        #     hits = hits + 1

        for fg in fgs:
            if fg in iupac:
                hits = hits + 1
            else:
                unhits = unhits + 1

    total = hits + unhits

    sr = hits/total
    sr = round(sr, 4)
    
    print('Total number is: ', total)
    print(f'File is {fname}, SR is: {sr}')


    unique_smiles = 0
    for smiles in smiles_list:
        if smiles not in train_smiles:
            unique_smiles = unique_smiles + 1

    print('Unique number is: ', unique_smiles/nums_smiles)

In [58]:
fname = './log/eval_fgs_5000000_f1_fg_5.log'
matrix_top(fname,5,5)

fname = './log/eval_fgs_5000000_f2_fg_5.log'
matrix_top(fname,5,5)

fname = './log/eval_fgs_5000000_f3_fg_5.log'
matrix_top(fname,5,5)

Num of iupac:  25000
Num of merged iupac:  2000
Num of smiles:  2000
Total number is:  2000
File is ./log/eval_fgs_5000000_f1_fg_5.log, SR is: 0.889
Unique number is:  0.991
Num of iupac:  25000
Num of merged iupac:  2000
Num of smiles:  2000
Total number is:  4000
File is ./log/eval_fgs_5000000_f2_fg_5.log, SR is: 0.9365
Unique number is:  0.9945
Num of iupac:  25000
Num of merged iupac:  2000
Num of smiles:  2000
Total number is:  6000
File is ./log/eval_fgs_5000000_f3_fg_5.log, SR is: 0.9395
Unique number is:  0.9855


In [68]:
fname = './log/eval_fgs_5000000_f1_fg_9.log'
matrix_top(fname,5,5)

fname = './log/eval_fgs_5000000_f2_fg_9.log'
matrix_top(fname,5,5)

fname = './log/eval_fgs_5000000_f3_fg_9.log'
matrix_top(fname,5,5)

Num of iupac:  25000
Num of merged iupac:  2000
Num of smiles:  2000
Total number is:  2000
File is ./log/eval_fgs_5000000_f1_fg_9.log, SR is: 0.918
Unique number is:  0.9875
Num of iupac:  25000
Num of merged iupac:  2000
Num of smiles:  2000
Total number is:  4000
File is ./log/eval_fgs_5000000_f2_fg_9.log, SR is: 0.9347
Unique number is:  0.995
Num of iupac:  25000
Num of merged iupac:  2000
Num of smiles:  2000
Total number is:  6000
File is ./log/eval_fgs_5000000_f3_fg_9.log, SR is: 0.9445
Unique number is:  0.9855


In [46]:
fname = './log/eval_fgs_13000000_f1_fg_0.log'
matrix_top(fname,5,1)

fname = './log/eval_fgs_13000000_f2_fg_0.log'
matrix_top(fname,5,1)

fname = './log/eval_fgs_13000000_f3_fg_0.log'
matrix_top(fname,5,1)

Num of iupac:  25000
Num of merged iupac:  2000
Total number is:  2000
File is ./log/eval_fgs_13000000_f1_fg_0.log, SR is: 0.7785
Num of iupac:  25000
Num of merged iupac:  2000
Total number is:  4000
File is ./log/eval_fgs_13000000_f2_fg_0.log, SR is: 0.8865
Num of iupac:  25000
Num of merged iupac:  2000
Total number is:  6000
File is ./log/eval_fgs_13000000_f3_fg_0.log, SR is: 0.869


In [3]:
fname = './log/eval_fgs_31000000_1_fg_4.log'
matrix_top(fname,1,1)

fname = './log/eval_fgs_31000000_2_fg_4.log'
matrix_top(fname,1,1)

fname = './log/eval_fgs_31000000_3_fg_4.log'
matrix_top(fname,1,1)

Num of iupac:  250000
Num of merged iupac:  2000
Total number is:  2000
File is ./log/eval_fgs_31000000_1_fg_4.log, SR is: 0.486
Num of iupac:  250000
Num of merged iupac:  2000
Total number is:  4000
File is ./log/eval_fgs_31000000_2_fg_4.log, SR is: 0.5627
Num of iupac:  250000
Num of merged iupac:  2000
Total number is:  6000
File is ./log/eval_fgs_31000000_3_fg_4.log, SR is: 0.5325


In [4]:
fname = './log/eval_fgs_31000000_1_fg_4.log'
matrix_top(fname,5,5)

fname = './log/eval_fgs_31000000_2_fg_4.log'
matrix_top(fname,5,5)

fname = './log/eval_fgs_31000000_3_fg_4.log'
matrix_top(fname,5,5)

Num of iupac:  250000
Num of merged iupac:  2000
Total number is:  2000
File is ./log/eval_fgs_31000000_1_fg_4.log, SR is: 0.4255
Num of iupac:  250000
Num of merged iupac:  2000
Total number is:  4000
File is ./log/eval_fgs_31000000_2_fg_4.log, SR is: 0.552
Num of iupac:  250000
Num of merged iupac:  2000
Total number is:  6000
File is ./log/eval_fgs_31000000_3_fg_4.log, SR is: 0.5393


In [5]:
fname = './log/eval_fgs_31000000_1_fg_4.log'
matrix_top(fname,25,5)

fname = './log/eval_fgs_31000000_2_fg_4.log'
matrix_top(fname,25,5)

fname = './log/eval_fgs_31000000_3_fg_4.log'
matrix_top(fname,25,5)

Num of iupac:  250000
Num of merged iupac:  2000
Total number is:  2000
File is ./log/eval_fgs_31000000_1_fg_4.log, SR is: 0.339
Num of iupac:  250000
Num of merged iupac:  2000
Total number is:  4000
File is ./log/eval_fgs_31000000_2_fg_4.log, SR is: 0.5397
Num of iupac:  250000
Num of merged iupac:  2000
Total number is:  6000
File is ./log/eval_fgs_31000000_3_fg_4.log, SR is: 0.5435


In [6]:
fname = './log/eval_fgs_31000000_1_fg_4.log'
matrix_top(fname,25,10)

fname = './log/eval_fgs_31000000_2_fg_4.log'
matrix_top(fname,25,10)

fname = './log/eval_fgs_31000000_3_fg_4.log'
matrix_top(fname,25,10)

Num of iupac:  250000
Num of merged iupac:  2000
Total number is:  2000
File is ./log/eval_fgs_31000000_1_fg_4.log, SR is: 0.4915
Num of iupac:  250000
Num of merged iupac:  2000
Total number is:  4000
File is ./log/eval_fgs_31000000_2_fg_4.log, SR is: 0.696
Num of iupac:  250000
Num of merged iupac:  2000
Total number is:  6000
File is ./log/eval_fgs_31000000_3_fg_4.log, SR is: 0.6888


In [7]:
fname = './log/eval_fgs_31000000_1_fg_4.log'
matrix_top(fname,25,15)

fname = './log/eval_fgs_31000000_2_fg_4.log'
matrix_top(fname,25,15)

fname = './log/eval_fgs_31000000_3_fg_4.log'
matrix_top(fname,25,15)

Num of iupac:  250000
Num of merged iupac:  2000
Total number is:  2000
File is ./log/eval_fgs_31000000_1_fg_4.log, SR is: 0.59
Num of iupac:  250000
Num of merged iupac:  2000
Total number is:  4000
File is ./log/eval_fgs_31000000_2_fg_4.log, SR is: 0.7815
Num of iupac:  250000
Num of merged iupac:  2000
Total number is:  6000
File is ./log/eval_fgs_31000000_3_fg_4.log, SR is: 0.7663


In [8]:
fname = './log/eval_fgs_31000000_1_fg_4.log'
matrix_top(fname,25,20)

fname = './log/eval_fgs_31000000_2_fg_4.log'
matrix_top(fname,25,20)

fname = './log/eval_fgs_31000000_3_fg_4.log'
matrix_top(fname,25,20)

Num of iupac:  250000
Num of merged iupac:  2000
Total number is:  2000
File is ./log/eval_fgs_31000000_1_fg_4.log, SR is: 0.6505
Num of iupac:  250000
Num of merged iupac:  2000
Total number is:  4000
File is ./log/eval_fgs_31000000_2_fg_4.log, SR is: 0.825
Num of iupac:  250000
Num of merged iupac:  2000
Total number is:  6000
File is ./log/eval_fgs_31000000_3_fg_4.log, SR is: 0.8118


In [9]:
fname = './log/eval_fgs_31000000_1_fg_4.log'
matrix_top(fname,25,25)

fname = './log/eval_fgs_31000000_2_fg_4.log'
matrix_top(fname,25,25)

fname = './log/eval_fgs_31000000_3_fg_4.log'
matrix_top(fname,25,25)

Num of iupac:  250000
Num of merged iupac:  2000
Total number is:  2000
File is ./log/eval_fgs_31000000_1_fg_4.log, SR is: 0.699
Num of iupac:  250000
Num of merged iupac:  2000
Total number is:  4000
File is ./log/eval_fgs_31000000_2_fg_4.log, SR is: 0.854
Num of iupac:  250000
Num of merged iupac:  2000
Total number is:  6000
File is ./log/eval_fgs_31000000_3_fg_4.log, SR is: 0.843
